# Model Card: Daily Dairy Stock-out-Date Predictor  

**Owner**: DS Team  
**Version**: 1.0
**Framework**: LightGBM 4.3  
**Objective**: Predict the number of days until first out-of-stock (OOS) for each dairy SKU at store level.

---

## Data
* **Source**: `features-daily_YYYY-MM_dairy.csv`
* **Period**: 2024-11-01 to 2024-11-30
* **Rows**: 54 740  
* **Features used** (15): `DailyBOH`, `boh_mean_3`, `boh_std_3`, `boh_mean_7`, …, `DayofWeek`, `is_holiday`, `itemsku` (categorical)

## Training & Evaluation
* **Split**: time-based – 11/01-25 train, 11/26-30 hold-out  
* **CV**: 5-fold `GroupKFold(itemsku)`
* **Metrics**  
  * CV MAE: **6.76 ± 0.29 days**  
  * Hold-out MAE: **3.55 days**

## Intended Use
* Daily batch generation of **stock-out risk dates** for replenishment planning.
* Not designed for intra-day (hourly) predictions.

## Limitations
* Single-month training – seasonality not yet captured.  
* SKUs that never went OOS are handled by a rule (`days_to_oos = 0`); model does not learn their behavior.  
* Holiday effects limited to US Thanksgiving; other holidays not encoded.

## Ethical & Operational Considerations
* False positives may trigger unnecessary replenishment; monitor MAE weekly.  
* Retraining recommendation: monthly, or whenever data distribution shifts >10 %.

## Changelog
* **v11** – Handoff LightGBM model trained on 2024-11 dairy data.

# Data Loading & Preprocessing

In [ ]:
# stockout_preprocess.py    
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- 0. Parameters ----------
DATA_PATH = Path("features-daily_2024-11_dairy.csv")

ROLL_WINDOWS = [3, 7]      # you can tweak later
HOLIDAYS = pd.to_datetime(["2024-11-28", "2024-11-29", "2024-11-30"])

# ---------- 1. Load ----------
df = pd.read_csv(DATA_PATH)
df["daydate"] = pd.to_datetime(df["daydate"])
print(f"Rows: {len(df):,} | Cols: {df.shape[1]}")

# ---------- 2. Basic EDA ----------
n_items = df["itemsku"].nunique()
print(f"Unique itemsku: {n_items:,}")

# identify items always zero
always_zero = (
    df.groupby("itemsku")["DailyBOH"]
      .apply(lambda s: (s == 0).all())
      .loc[lambda s: s]
      .index.tolist()
)
print(f"Items always zero in Nov: {len(always_zero):,}")

# ---------- 2b. Save permanent OOS list ----------
import json
PERM_OOS_PATH = Path("models/prod_v1/permanent_oos_list.json")

always_zero = (
    df.groupby("itemsku")["DailyBOH"]
      .apply(lambda s: (s == 0).all())
      .loc[lambda s: s]
      .index
      .astype(str)            # convert to string for consistency
      .tolist()
)

PERM_OOS_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(PERM_OOS_PATH, "w") as f:
    json.dump({"generated_on": str(pd.Timestamp.today().date()),
               "sku_count": len(always_zero),
               "itemsku":   always_zero},
              f, indent=2)

print(f"Permanent OOS list saved → {PERM_OOS_PATH}  ({len(always_zero)} SKUs)")

# ---------- 3. Holiday flag ----------
df["is_holiday"] = df["daydate"].isin(HOLIDAYS).astype(int)

# ---------- 4. Create stock-out label ----------
def first_zero_date(s):
    zeros = s.index[s == 0]
    return zeros.min() if len(zeros) else pd.NaT

stockout_map = (
    df.groupby("itemsku")
      .apply(lambda g: first_zero_date(g.set_index("daydate")["DailyBOH"]))
)
df = df.merge(
    stockout_map.rename("oos_date"),
    on="itemsku",
    how="left"
)

# days_to_oos (positive integer; NaN if never stock-out within window)
df["days_to_oos"] = (df["oos_date"] - df["daydate"]).dt.days
df.loc[df["days_to_oos"] < 0, "days_to_oos"] = np.nan  # after OOS date → ignore

# ---------- 5. Rolling features ----------
df_sorted = df.sort_values(["itemsku", "daydate"])
for win in ROLL_WINDOWS:
    df_sorted[f"boh_mean_{win}"] = (
        df_sorted.groupby("itemsku")["DailyBOH"]
                 .transform(lambda x: x.rolling(win, min_periods=1).mean())
    )
    df_sorted[f"boh_std_{win}"] = (
        df_sorted.groupby("itemsku")["DailyBOH"]
                 .transform(lambda x: x.rolling(win, min_periods=1).std().fillna(0))
    )

# ---------- 6. Train / Test split ----------
train_end = pd.Timestamp("2024-11-25")
train_mask = df_sorted["daydate"] <= train_end
test_mask  = df_sorted["daydate"] > train_end

train_df = df_sorted[train_mask & df_sorted["days_to_oos"].notna() & ~df_sorted["itemsku"].isin(always_zero)]
test_df  = df_sorted[test_mask]

print(f"Train rows: {len(train_df):,} | Test rows: {len(test_df):,}")
print(f"Memory usage  (train): {train_df.memory_usage(deep=True).sum()/1e6:.2f} MB")

# Optional: save processed parquet for faster reload
train_df.to_parquet("train_proc.parquet", index=False)
test_df.to_parquet("test_proc.parquet", index=False)

Rows: 54,740 | Cols: 39
Unique itemsku: 1,247
Items always zero in Nov: 212
Permanent OOS list saved → models/prod_v1/permanent_oos_list.json  (212 SKUs)


/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/ipykernel_35964/1677873271.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: first_zero_date(g.set_index("daydate")["DailyBOH"]))


Train rows: 4,299 | Test rows: 8,620
Memory usage  (train): 5.04 MB


# Model Building

# [LightGBM] Model Building: Stockout Predictor

In [ ]:
# train_lgbm_stockout.py    
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, median_absolute_error
from sklearn.model_selection import GroupKFold, cross_val_score
from lightgbm import LGBMRegressor
import joblib
import yaml

# ---------- 0. Parameters ----------
TRAIN_PATH = Path("train_proc.parquet")
TEST_PATH  = Path("test_proc.parquet")
MODEL_OUT  = Path("lgbm_stockout_model.txt")

# ---------- 1. Load ----------
train_df = pd.read_parquet(TRAIN_PATH)
test_df  = pd.read_parquet(TEST_PATH)

# permanent OOS list (saved during preprocessing)
permanent_oos_list = (
    train_df["itemsku"].unique().tolist() +
    test_df.loc[test_df["DailyBOH"] == 0, "itemsku"].unique().tolist()
)
permanent_oos_list = list(set(permanent_oos_list))

# ---------- 2. Feature selection ----------
EXCLUDE = {
    "daydate", "oos_date", "days_to_oos"
}
num_feats = [c for c in train_df.columns
             if c not in EXCLUDE and
                train_df[c].dtype != "object" and
                not c.startswith("itemsku")]

CAT_FEATS = ["itemsku"]

FEATURES = num_feats + CAT_FEATS

# ensure categorical dtype
for c in CAT_FEATS:
    train_df[c] = train_df[c].astype("category")
    test_df[c]  = test_df[c].astype("category")

X = train_df[FEATURES]
y = train_df["days_to_oos"]

# ---------- 3. Model ----------
lgbm_params = dict(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    metric="mae",
)

model = LGBMRegressor(**lgbm_params)

# ---------- 4. Cross-validation ----------
gkf = GroupKFold(n_splits=5)
cv_mae = -cross_val_score(
    model, X, y,
    cv=gkf.split(X, y, groups=train_df["itemsku"]),
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
)
print(f"CV MAE (5-fold, grouped by itemsku): {cv_mae.mean():.2f} ± {cv_mae.std():.2f}")

# ---------- 5. Fit on full train ----------
model.fit(
    X, y,
    categorical_feature=CAT_FEATS,
)

# ---------- 6. Evaluate on hold-out (11/26–30) ----------
test_eval = test_df[test_df["days_to_oos"].notna()].copy()

if len(test_eval) == 0:
    print("⚠️  No stock-out events in 11/26–11/30 — skipping hold-out MAE.")
    overall_mae = overall_medae = None
else:
    test_eval["pred"] = model.predict(test_eval[FEATURES])
    overall_mae   = mean_absolute_error(test_eval["days_to_oos"], test_eval["pred"])
    overall_medae = median_absolute_error(test_eval["days_to_oos"], test_eval["pred"])
    print(f"Hold-out MAE  : {overall_mae:.2f} days")
    print(f"Hold-out MedAE: {overall_medae:.2f} days")

    # per-item MAE distribution
    per_item = (
        test_eval.groupby("itemsku")
                 .apply(lambda g: np.mean(np.abs(g["days_to_oos"] - g["pred"])))
                 .rename("item_mae")
    )
    print(per_item.describe(percentiles=[0.5, 0.75, 0.9]))

# ---------- 6b. Feature Importance (after model.fit) ----------
# import matplotlib.pyplot as plt

# imp_df = pd.DataFrame({
#     "feature": model.feature_name_,
#     "gain":    model.booster_.feature_importance(importance_type="gain"),
#     "split":   model.booster_.feature_importance(importance_type="split")
# }).sort_values("gain", ascending=False)

# print(imp_df.head(20))   # Top 20

# ---------- 6c. Plotting ----------
# topN = 15
# imp_df.head(topN).plot.barh(
#     x="feature", y="gain", figsize=(6, 5), legend=False
# )
# plt.gca().invert_yaxis()
# plt.title("Top Feature Gain")
# plt.tight_layout()
# plt.show()

# ---------- 7. Save model ----------
model.booster_.save_model(str(MODEL_OUT))
print(f"Model saved to {MODEL_OUT.resolve()}")

# ---------- 8. Save params.yaml ----------
def to_python(obj):
    """Recursively convert numpy scalars / arrays to builtin Python types."""
    if isinstance(obj, (np.floating, np.integer)):
        return obj.item()                    # -> float or int
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, dict):
        return {k: to_python(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_python(v) for v in obj]
    return obj                               # str, bool, None, etc.

PARAMS_PATH = Path("models/prod_v1/params.yaml")
PARAMS_PATH.parent.mkdir(parents=True, exist_ok=True)

params_dict = {
    "generated_on": str(pd.Timestamp.today()),
    "framework":    "LightGBM 4.3",
    "hyperparams":  lgbm_params,
    "categorical_features": CAT_FEATS,
    "numeric_features":     num_feats,
    "cv_mae_mean":  float(cv_mae.mean()),
    "cv_mae_std":   float(cv_mae.std()),
    "holdout_mae":  None if overall_mae is None else float(overall_mae),
    "holdout_medae": None if overall_medae is None else float(overall_medae),
    "training_rows": int(len(train_df)),
    "training_period": "2024-11-01 → 2024-11-25"
}

with open(PARAMS_PATH, "w") as f:
    yaml.safe_dump(to_python(params_dict), f, sort_keys=False)

print(f"Training parameters saved → {PARAMS_PATH}")

# ---------- 8. Inference helper ----------
def predict_days_to_oos(sample_df):
    """
    Input  : DataFrame with same columns as `train_df`
    Output : Series of predicted days_to_oos
    Rule   : If itemsku in permanent_oos_list => 0
    """
    sample_df = sample_df.copy()
    sample_df["itemsku"] = sample_df["itemsku"].astype("category")
    pred = model.predict(sample_df[FEATURES])
    pred = np.maximum(pred, 0)  # no negative days
    pred = np.round(pred).astype(int)
    pred[sample_df["itemsku"].isin(permanent_oos_list)] = 0
    return pd.Series(pred, index=sample_df.index, name="pred_days_to_oos")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002454 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2141
[LightGBM] [Info] Number of data points in the train set: 3439, number of used features: 27
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003411 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2098
[LightGBM] [Info] Total Bins 2145
[LightGBM] [Info] Number of data points in the train set: 3439, number of used features: 27
[LightGBM] [Info] Number of data poi

/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/ipykernel_35964/3180274867.py:94: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_eval.groupby("itemsku")
/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/ipykernel_35964/3180274867.py:95: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.mean(np.abs(g["days_to_oos"] - g["pred"])))
